In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets/data-prep-assignment/git_web_ml

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets/data-prep-assignment/git_web_ml


In [6]:
%ls

citing.txt  musae_git_edges.csv  musae_git_features.json  musae_git_target.csv  README.txt


In [8]:
import pandas as pd
import json

# Load edges data
edges_df = pd.read_csv('musae_git_edges.csv')

# Load target data
target_df = pd.read_csv('musae_git_target.csv')

# Load features data
with open('musae_git_features.json', 'r') as file:
    features_data = json.load(file)

# Convert the features data into a DataFrame
features_df = pd.DataFrame.from_dict(features_data, orient='index')

edges_df.head()

,id_1,id_2
0,0,23977
1,1,34526
2,1,2370
3,1,14683
4,1,29982


In [9]:
target_df.head()

,id,name,ml_target
0,0,Eiryyy,0
1,1,shawflying,0
2,2,JpMCarrilho,1
3,3,SuhwanCha,0
4,4,sunilangadi2,1


In [11]:
features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,1574,3773,3571,2672,2478,2534,3129,3077,1171.0,2045.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1193,376,73,290,3129,1852,3077,1171,1022.0,2045.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1574,3773,925,1728,2815,2963,3077,364,1171.0,536.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3964,3773,4003,928,1852,3077,364,1022,3763.0,2045.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1929,3773,1793,3511,1290,3129,3077,364,1171.0,1022.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The anonomized features are not standarized. We must standardize them


In [15]:
# Define a function to compute aggregation features for each entry
def compute_aggregated_features(entry):
    if isinstance(entry, list):  # If the entry is a list or array
        return [sum(entry) / len(entry),   # Mean
                sorted(entry)[len(entry) // 2],  # Median
                (max(entry) - min(entry)) / 2,  # Range/2 as a proxy for Standard Deviation
                min(entry),  # Minimum
                max(entry)]  # Maximum
    else:  # If the entry is a single number
        return [entry, entry, 0, entry, entry]

# Apply the function to each row in the features DataFrame
aggregated_features_df = features_df.applymap(compute_aggregated_features)

# Split the aggregated features into separate columns
aggregated_features_df[['Mean', 'Median', 'Range/2', 'Min', 'Max']] = pd.DataFrame(aggregated_features_df[0].tolist(), index=aggregated_features_df.index)
aggregated_features_df = aggregated_features_df.drop(0, axis=1)

aggregated_features_df.head()




,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,Mean,Median,Range/2,Min,Max
0,"[3773, 3773, 0, 3773, 3773]","[3571, 3571, 0, 3571, 3571]","[2672, 2672, 0, 2672, 2672]","[2478, 2478, 0, 2478, 2478]","[2534, 2534, 0, 2534, 2534]","[3129, 3129, 0, 3129, 3129]","[3077, 3077, 0, 3077, 3077]","[1171.0, 1171.0, 0, 1171.0, 1171.0]","[2045.0, 2045.0, 0, 2045.0, 2045.0]","[1539.0, 1539.0, 0, 1539.0, 1539.0]",...,"[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]",1574,1574,0,1574,1574
1,"[376, 376, 0, 376, 376]","[73, 73, 0, 73, 73]","[290, 290, 0, 290, 290]","[3129, 3129, 0, 3129, 3129]","[1852, 1852, 0, 1852, 1852]","[3077, 3077, 0, 3077, 3077]","[1171, 1171, 0, 1171, 1171]","[1022.0, 1022.0, 0, 1022.0, 1022.0]","[2045.0, 2045.0, 0, 2045.0, 2045.0]","[536.0, 536.0, 0, 536.0, 536.0]",...,"[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]",1193,1193,0,1193,1193
2,"[3773, 3773, 0, 3773, 3773]","[925, 925, 0, 925, 925]","[1728, 1728, 0, 1728, 1728]","[2815, 2815, 0, 2815, 2815]","[2963, 2963, 0, 2963, 2963]","[3077, 3077, 0, 3077, 3077]","[364, 364, 0, 364, 364]","[1171.0, 1171.0, 0, 1171.0, 1171.0]","[536.0, 536.0, 0, 536.0, 536.0]","[1867.0, 1867.0, 0, 1867.0, 1867.0]",...,"[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]",1574,1574,0,1574,1574
3,"[3773, 3773, 0, 3773, 3773]","[4003, 4003, 0, 4003, 4003]","[928, 928, 0, 928, 928]","[1852, 1852, 0, 1852, 1852]","[3077, 3077, 0, 3077, 3077]","[364, 364, 0, 364, 364]","[1022, 1022, 0, 1022, 1022]","[3763.0, 3763.0, 0, 3763.0, 3763.0]","[2045.0, 2045.0, 0, 2045.0, 2045.0]","[3859.0, 3859.0, 0, 3859.0, 3859.0]",...,"[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]",3964,3964,0,3964,3964
4,"[3773, 3773, 0, 3773, 3773]","[1793, 1793, 0, 1793, 1793]","[3511, 3511, 0, 3511, 3511]","[1290, 1290, 0, 1290, 1290]","[3129, 3129, 0, 3129, 3129]","[3077, 3077, 0, 3077, 3077]","[364, 364, 0, 364, 364]","[1171.0, 1171.0, 0, 1171.0, 1171.0]","[1022.0, 1022.0, 0, 1022.0, 1022.0]","[2045.0, 2045.0, 0, 2045.0, 2045.0]",...,"[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]","[nan, nan, 0, nan, nan]",1929,1929,0,1929,1929


In [18]:
# Drop the original varying features and retain only the aggregated ones
aggregated_features_df = aggregated_features_df[['Mean', 'Median', 'Range/2', 'Min', 'Max']]

aggregated_features_df

,Mean,Median,Range/2,Min,Max
0,1574,1574,0,1574,1574
1,1193,1193,0,1193,1193
2,1574,1574,0,1574,1574
3,3964,3964,0,3964,3964
4,1929,1929,0,1929,1929
...,...,...,...,...,...
37695,1574,1574,0,1574,1574
37696,1929,1929,0,1929,1929
37697,3433,3433,0,3433,3433
37698,3730,3730,0,3730,3730


In [19]:
# Reset the index to add a unique identifier to each row
aggregated_features_df = aggregated_features_df.reset_index()
aggregated_features_df = aggregated_features_df.rename(columns={"index": "user_id"})

aggregated_features_df.head()

,user_id,Mean,Median,Range/2,Min,Max
0,0,1574,1574,0,1574,1574
1,1,1193,1193,0,1193,1193
2,2,1574,1574,0,1574,1574
3,3,3964,3964,0,3964,3964
4,4,1929,1929,0,1929,1929


Check for duplicates

In [20]:
# Check for duplicate rows in each DataFrame
edges_duplicates = edges_df.duplicated().sum()
target_duplicates = target_df.duplicated().sum()
features_duplicates = aggregated_features_df.duplicated().sum()

edges_duplicates, target_duplicates, features_duplicates

(0, 0, 0)

In [22]:
!pip install torch torchvision
!pip install torch-geometric \
  -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.8 MB/s eta 0:00:00


In [24]:
target_df

,id,name,ml_target
0,0,Eiryyy,0
1,1,shawflying,0
2,2,JpMCarrilho,1
3,3,SuhwanCha,0
4,4,sunilangadi2,1
...,...,...,...
37695,37695,shawnwanderson,1
37696,37696,kris-ipeh,0
37697,37697,qpautrat,0
37698,37698,Injabie3,1


Convert Df's into tensor

In [29]:
import torch
from torch_geometric.data import Data

# Convert edges to tensor
edge_index = torch.tensor(edges_df.values, dtype=torch.long).t().contiguous()

# Convert node features to tensor
x = torch.tensor(aggregated_features_df.drop('user_id', axis=1).values, dtype=torch.float)

# Convert labels to tensor
# Assuming 'ml_developer' is 1 and 'web_developer' is 0
y = torch.tensor(target_df['ml_target'].astype(int).values, dtype=torch.long)

# Create a PyTorch Geometric data object
data = Data(x=x, edge_index=edge_index, y=y)

# Assuming a mask for training data
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[:int(0.8 * data.num_nodes)] = 1  # 80% of data for training

# Mask for testing data
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask[int(0.8 * data.num_nodes):] = 1  # 20% of data for testing

Model Definition

In [30]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(data.num_features, 2)

Train Model

In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Assuming a mask for training data is available
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[:int(0.8 * data.num_nodes)] = 1  # 80% of data for training

for epoch in range(100):
    loss = train(data)
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 242.54327392578125
Epoch 2, Loss: 166.8780517578125
Epoch 3, Loss: 94.7803955078125
Epoch 4, Loss: 26.2241153717041
Epoch 5, Loss: 127.39324951171875
Epoch 6, Loss: 125.73772430419922
Epoch 7, Loss: 59.12345886230469
Epoch 8, Loss: 12.439874649047852
Epoch 9, Loss: 33.54753875732422
Epoch 10, Loss: 47.00498962402344
Epoch 11, Loss: 54.1783332824707
Epoch 12, Loss: 56.17488479614258
Epoch 13, Loss: 53.92321014404297
Epoch 14, Loss: 48.21147537231445
Epoch 15, Loss: 39.70656967163086
Epoch 16, Loss: 28.964231491088867
Epoch 17, Loss: 16.435346603393555
Epoch 18, Loss: 2.474257707595825
Epoch 19, Loss: 41.34044647216797
Epoch 20, Loss: 52.452545166015625
Epoch 21, Loss: 34.69226837158203
Epoch 22, Loss: 1.596938133239746
Epoch 23, Loss: 10.18776798248291
Epoch 24, Loss: 15.566906929016113
Epoch 25, Loss: 18.102981567382812
Epoch 26, Loss: 18.13152313232422
Epoch 27, Loss: 15.950227737426758
Epoch 28, Loss: 11.824624061584473
Epoch 29, Loss: 5.990290641784668
Epoch 30, Loss:

In [32]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
accuracy = correct / int(data.test_mask.sum())
print('Accuracy:', accuracy)

Accuracy: 0.7396551724137931
